# Popy

In agent-based models, the network structure is a key element that needs to be modeled with care.
When empirical network data are not available, simulation models must rely on network models.
Popy is a new way to quickly create network models ready to use in agent-based models.

Popy relies on a concept for the generation of network models which is currently mainly used in epidemiological models: the definition of so-called contact layers.
In Popy those contact layers are simply called *locations*.
*Locations* are places where agents can meet.
Thus, if agents are at the same location at the same time, they can interact.
...

# Creating static networks using Popy

In the following, we want to recreate the contact network of a school.
Assume we have access to the following sample of individual data collected in a school.
We use this data as a starting point for our network.

In [1]:
import popy
from popy.pop_maker import PopMaker
import popy.utils as utils

import pandas as pd

In [2]:
df_school = pd.read_csv("example_school_data.csv", sep=";")
df_school.head()

,status,gender,grade,hours,friend_group
0,pupil,m,1.0,4,1
1,pupil,w,1.0,4,1
2,pupil,m,1.0,4,2
3,pupil,w,1.0,4,2
4,pupil,m,2.0,4,3


In [3]:
df_school.tail()

,status,gender,grade,hours,friend_group
37,teacher,w,NaN,9,0
38,teacher,m,NaN,10,0
39,teacher,w,NaN,5,0
40,social_worker,w,NaN,6,0
41,social_worker,w,NaN,4,0


## The first location

Lets start with the definition of our first location: a class room.
We start by just using the default location class without defining any further attributes or methods.
Every location class must inherit - directly or indirectly - from `popy.Location`.

In [4]:
# Define a Location class
class ClassRoom(popy.Location):
    pass

Using the `PopMaker`, we can create both the population of agents and the location objects.
If we are only interested in the network, only the agents are relevant to us, as they contain all the relevant network information.
If we want to build a simulation model, the locations become relevant, too.
For now, we only focus on creating static networks and, thus, ignore the created locations.

In [5]:
# create a PopMaker-Object
pop_maker = PopMaker()

# Let the PopMaker create agents and locations
agents, locations = pop_maker.make(
    df=df_school,
    location_classes=[ClassRoom],
)

One of best ways to explore and validate the resulting network is to visualize it:

In [6]:
utils.plot_network(agents=agents, node_attrs=df_school.columns)

The network graph shows a fully connected network.
Note that only the agents are shown as nodes in the network diagram.
Locations are only shown implicitly in the edges between the agents.
Every default location creates a fully connected network at the agent level, as in the bipartite graph every agent is assigned to this default location.


## Setting the size

In the next step, we specify the number of people in one classroom.
In this example, we assume tiny classrooms of four agents.
To set the number of agents per location, we need to overwrite the `setup()`-method.
Within the `setup()`-method we can set the number of agents by setting `self.size` to the desired value.

In [7]:
class ClassRoom(popy.Location):
    # Overwrite the method setup() to set the number of agents (and many others)
    def setup(self):
        self.size = 4

In [8]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns)

The network diagramm above now shows multiple clusters of size around 4.
Each cluster represents one class room.
If we set a specific size for a location, as many location instances with that size as needed are created and agents are randomly assigned to one of these location instances.
As you can see, some classrooms have more than four members, because the number of agents that are assigned to classrooms cannot be divided exactly by the desired size of four.
If you want to make sure that no classroom has more than four members, set the location attribute `allow_overcrowding` to `False`:

In [9]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
        self.allow_overcrowding = False

In [10]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns)

## Specifying location visitors

The classrooms above are made of all agents.
But in many cases we want specific locations to be exclusively accessible to certain agents.
For this scenario the method `filter()` exists.
If this method returns `True`, an agent gets assigned to an instance of this location class.
The most common way to use this method is to specify a condition that requires a certain agent attribute to contain a certain value.

In this example we want classrooms to be only accessible for pupils.

In [11]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"

In [12]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

Now classrooms consist only of pupils (orange nodes), while all other agents do not belong to any location.

## Building separated locations

In [13]:
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

The above network diagram, where the nodes are colored by grade, shows that the classrooms are not separated by grade.
To seperate agents by grade, we could define one location class for each grade and use `filter()` to assign only agents with a specific grade value to a specific location.

However, a more convenient way to do it is to use the method `split()`.
For each agent, the method `split()` returns one value.
For each unique value seperated location instances are created.

In [14]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade

In [15]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

## Keeping agents together

The following plot shows that the members of one friend group are distributed over different classrooms.

In [16]:
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="friend_group")

Although this is a very realistic situation, in this example, we assume that all friend group members are always in the same class.
To implement that, we use the location method `stick_together()`:
For each agent, the method `stick_together()` returns a specific value.
Agents with the same return value are sticked together.

In [17]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

In [18]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="friend_group")

## Edge weights

Until now, all edges between nodes have a weight of `1`.
The location method `weight()` can be used to set different weights.
In the following, we set the weight of all edges generated by a classroom to `5`.
This number could, for instance, represent that agents are together in classrooms for five hours.

In [19]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return 5

In [20]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

To implement agent-specific weights, we could also let `weight()` return an agent attribute. 
In this case we use the agent attribute `agent.hours`:

In [21]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    

pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

As the value returned by `location.weight()` refers to the weight between the agent and the location, all the weights between the agents and the locations must be somehow combined when determining the weight between agents (aka graph projection).
The location method `project_weights()` defines how those weights are combined.
By default, `project_weights()` returns the smallest weight of the two to be combined.
The code cell below shows how `project_weights()` combines the two weights by default.
In this example, we keep this way of combining the weights, but this method could be easily rewritten.

In [22]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

In [23]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

## Bringing together different agents

So far, we are able to connect agents who share a certain attribut value.
But what if we want to explicitly connect agents who have different values on a certain attribute?
One solution could be to simply give those agents we want to be in the same location the same value on a certain attribute and then define a location that brings together these agents.

Another possibility is to *melt* different locations into one location.
To do this, we have to define at least three locations:  Two or more locations that are the components that get melted into one location and one location that melts those components together.

Assume we want to create classrooms that consist of one teacher and four pupils.
To create such a location, we first define a location (`TeachersInClassRoom`) that consists of only one teacher.
Then we define a location (`PupilsInClassRoom`) that consists of four pupils.
Finally, we define a location (`ClassRoom`) that uses the method `melt()` to melt the two previously defined locations into one location.
The method `melt()` must return a tuple or list of at least two location classes that shall be melted into one.

In [24]:
# a location for teachers
class TeachersInClassRoom(popy.Location):
    def setup(self):
        self.size = 1

    def filter(self, agent):
        return agent.status == "teacher"

# a location for pupils
class PupilsInClassRoom(popy.Location):
    def setup(self):
        self.size = 4

    def filter(self, agent):
        return agent.status == "pupil"

# a location for teachers and pupils
class ClassRoom(popy.Location):
    def melt(self):
        return TeachersInClassRoom, PupilsInClassRoom

In [25]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

Now we bring back all other settings we made so far:

In [26]:
class TeachersInClassRoom(popy.Location):
    def setup(self):
        self.size = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.Location):
    def setup(self):
        self.size = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.Location):
    def melt(self):
        return TeachersInClassRoom, PupilsInClassRoom
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])
    
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

## More than one location

The melting of locations combines different locations into one location, but does not create multiple different location classes.
If we want to generate multiple different location types, we have to simply feed more than one location class into the `PopMaker`.

In the following, we introduce a `School` as a second type of location.
Each school has about `20` members.
(In order to keep the code clearer, we skip the merging of locations temporarely.)

In [27]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])


class School(popy.Location):
    def setup(self):
        self.size = 20

pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom, School])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

## Nesting locations
The plot above shows two big clusters.
Each of those clusters represents one school.
The plot above also shows something unrealistic: The schools are connected because members of one class are not always in the same school.
We can use the `School`-method `stick_together()` to fix this issue.
This works because each agent gets a new attribute named after the location class and is set to a location identifier when assigned to an instance of this location class.
This attribute can be used afterwards to define the structure of other locations.

In [28]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

class School(popy.Location):
    def setup(self):
        self.size = 22

    def stick_together(self, agent):
        return agent.ClassRoom


pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom, School])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

However, it is important that the agents get assigned to the classrooms before getting assigned to the schools.
That means, in general, the order of the creation of the locations is important and, hence, the order of the location classes given to the `location_classes` argument.

If we build schools before classrooms, the above method of sticking together classrooms does not work anymore:

In [29]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[School, ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

As we saw, the method `stick_together()` can be used to nest locations into other locations.
However, this approach is limited because we can only specify one location class as the return value of `stick_together()`.

Another way to nest locations into other locations is to use the location method `nest()`.
The method `nest()` can return `None` or a location class.
If `nest()` returns a location class, the location is nested into the returned location class.
For instance, to nest classrooms within schools, we must define the method `nest()` for the location `ClassRoom` and let this method return `School`.
Again, the order of location creation plays a crucial role: The level-1 location must always be created after the level-2 location.

In [30]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])
    
    def nest(self):
        return School

class School(popy.Location):
    def setup(self):
        self.size = 20


pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[School, ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

`nest()` allows us to nest as many locations in as many levels as we want.
However, `nest()` has one disadvantage: Because the agents are first grouped into the level-2 location and then into the level-1 location, specific compositions defined at level 1 are not considered when grouping the agents into the level-2 locations.

The following example demonstrates that:

In [31]:
class TeachersInClassRoom(popy.Location):
    def setup(self):
        self.size = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.Location):
    def setup(self):
        self.size = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
class ClassRoom(popy.Location):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
    def weight(self, agent):
        return agent.hours * 10
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])
    
    def nest(self):
        return School

class School(popy.Location):
    def setup(self):
        self.size = 20
    
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[School, ClassRoom])
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

As you can see in the graph above, the compositions defined by `ClassRoom` are not always met.
That is due to the fact that the agents are first assigned to the schools independently of the settings defined by `ClassRoom`.
So when the classrooms are built, there are not always the *necessary* agents in each school needed to meet the composition defined in `ClassRoom`.
This might not be always a problem.
However, if we want to ensure that classrooms always have the defined composition of agents, they have to be created before the schools are created and, thus, we have to use the method `stick_together()` to nest classrooms into schools:

In [32]:
class TeachersInClassRoom(popy.Location):
    def setup(self):
        self.size = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.Location):
    def setup(self):
        self.size = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.Location):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
    def weight(self, agent):
        return agent.hours * 10
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

class School(popy.Location):
    def setup(self):
        self.size = 20
    
    def stick_together(self, agent):
        return agent.ClassRoom
    
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom, School])
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

It is also possible to combine both nesting techniques.
We could first nest the class `ClassRoom` into `School` using `stick_together()` in order to get the composition wanted for `ClassRoom` and then use `nest()` to nest further locations into `School`, as it is done in the nest example:

In [33]:
class TeachersInClassRoom(popy.Location):
    def setup(self):
        self.size = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.Location):
    def setup(self):
        self.size = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.Location):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
    def weight(self, agent):
        return agent.hours * 10
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

class School(popy.Location):
    def setup(self):
        self.size = 20
    
    def stick_together(self, agent):
        return agent.ClassRoom

class SoccerTeam(popy.Location):
    def setup(self):
        self.size = 11
    
    def nest(self):
        return School
    
pop_maker = PopMaker()
agents, locations = pop_maker.make(
    df=df_school, 
    location_classes=[
        ClassRoom, # nested into School using `stick_together()`
        School, 
        SoccerTeam, # nested into School using `nest()`
        ]
    )
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

This way we can nest multiple locations into one level-2 location and at the same time ensure that the composition of agents defined in `ClassRoom` is met in each school.
The order of the locations plays a crucial role:
1. The classrooms with the desired compositions are created
2. The schools are created keeping together whole classrooms
3. Soccer teams are created within the schools.

## More ways to inspect the network

Max's Funktionen...

In [34]:
utils.network_measures(agent_list=agents)

[{'diameter': 3,
  'density': 1.0,
  'transitivity': 1.0,
  'avg_clustering': 0.0794103397432602,
  'avg_path_length': 1.731578947368421},
 {'diameter': 3,
  'density': 1.0,
  'transitivity': 1.0,
  'avg_clustering': 0.06313436231132091,
  'avg_path_length': 1.6493506493506493}]